# **Feature Engineering - POS_CASH_balance**

## **Lendo dados com Spark**

### **Importar Bibliotecas**

In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1701607570197_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import os
import sys
import pytz
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### **Importar tabela 'POS_CASH_balance'.**

In [3]:
spark = SparkSession.builder.appName('Feature Engineering').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt1/yarn/usercache/livy/appcache/application_1701607570197_0001/container_1701607570197_0001_01_000001/pyspark.zip/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.

In [4]:
file='POS_CASH_balance.csv'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
S3_DATA_SOURCE_PATH = 'pod-academy-analise-de-credito-para-fintech/dados'
S3_DATA_OUTPUT_PATH = 'pod-academy-analise-de-credito-para-fintech/feature-engineering/02-Previous-Application'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
output = 'abt_POS_CASH_balance'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
full_path_source = 's3://{bkt}/{file}'.format(bkt=S3_DATA_SOURCE_PATH,file=file)
print(full_path_source)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://pod-academy-analise-de-credito-para-fintech/dados/POS_CASH_balance.csv

In [8]:
POS_CASH_balance_00 = spark.read.format(
       "com.databricks.spark.csv").option(
       "header", "true").option(
       "encoding", "ISO-8859-1").option(
       "encoding", "UTF-8").option(
       "inferSchema", "false").option(
        "delimiter", ',').load(
       full_path_source)
POS_CASH_balance_00.registerTempTable("POS_CASH_balance_00")
POS_CASH_balance_00.cache()
qtd=POS_CASH_balance_00.count()
print('records ingestion data:', qtd)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

records ingestion data: 10001358
/mnt1/yarn/usercache/livy/appcache/application_1701607570197_0001/container_1701607570197_0001_01_000001/pyspark.zip/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.

In [9]:
POS_CASH_balance_00.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|CNT_INSTALMENT|CNT_INSTALMENT_FUTURE|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|
+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+
|   1803195|    182943|           -31|          48.0|                 45.0|              Active|     0|         0|
|   1715348|    367990|           -33|          36.0|                 35.0|              Active|     0|         0|
|   1784872|    397406|           -32|          12.0|                  9.0|              Active|     0|         0|
|   1903291|    269225|           -35|          48.0|                 42.0|              Active|     0|         0|
|   2341044|    334279|           -35|          36.0|                 35.0|              Active|     0|         0|
|   2207092|    342166|           -32|          12.0|                 12.0|     

In [10]:
POS_CASH_balance_00.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- SK_ID_PREV: string (nullable = true)
 |-- SK_ID_CURR: string (nullable = true)
 |-- MONTHS_BALANCE: string (nullable = true)
 |-- CNT_INSTALMENT: string (nullable = true)
 |-- CNT_INSTALMENT_FUTURE: string (nullable = true)
 |-- NAME_CONTRACT_STATUS: string (nullable = true)
 |-- SK_DPD: string (nullable = true)
 |-- SK_DPD_DEF: string (nullable = true)

### **Comentário**

- Cada "SK_ID_PREV" se refere a um único "SK_ID_CURR".

- Um "SK_ID_CURR" pode corresponder a diferentes "SK_ID_PREV".

- Ou seja, podemos referenciar as variáveis criadas com a tabela "application", assim como, a tabela "previous_application".

In [11]:
# POS_CASH_balance_00 = spark.sql('''
# SELECT
#   *
# FROM
#   POS_CASH_balance
# WHERE
#   SK_ID_CURR = '405549'
# ''')

# POS_CASH_balance_00.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# # Contar valores distintos da variável SK_ID_CURR
# qtde_distinct_sk_id_curr = spark.sql('''
# SELECT
#   COUNT(DISTINCT SK_ID_CURR) AS qtde_distinct_sk_id_curr
# FROM
#   POS_CASH_balance
# ''')

# # Exibir a quantidade de valores distintos
# qtde_distinct_sk_id_curr.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# **Criação de variáveis preditivas**

## **POS_CASH_balance**

In [13]:
# Definir funções de agregações desejadas
agg = ['SUM', 'MAX', 'MIN', 'AVG']

var_categoricas = ['NAME_CONTRACT_STATUS']
var_numericas = ['CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE', 'MONTHS_BALANCE', 'SK_DPD', 'SK_DPD_DEF']

# Definir flags de meses
meses_flag = ['ultimos_3_meses_flag', 'ultimos_6_meses_flag', 'ultimos_9_meses_flag', 'ultimos_12_meses_flag']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
POS_CASH_balance_00.createOrReplaceTempView('POS_CASH_balance_00')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## **Criando flags de janela para histórico:**

- últimos 3 meses

- últimos 6 meses

- últimos 9 meses

- últimos 12 meses

In [15]:
POS_CASH_balance_01 = spark.sql("""
SELECT
    *,
    CASE
        WHEN MONTHS_BALANCE BETWEEN MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_CURR`) - 3 AND MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_CURR`) THEN 1
        ELSE 0
    END AS ultimos_3_meses_flag,
    CASE
        WHEN MONTHS_BALANCE BETWEEN MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_CURR`) - 6 AND MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_CURR`) THEN 1
        ELSE 0
    END AS ultimos_6_meses_flag,
    CASE
        WHEN MONTHS_BALANCE BETWEEN MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_CURR`) - 6 AND MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_CURR`) THEN 1
        ELSE 0
    END AS ultimos_9_meses_flag,
    CASE
        WHEN MONTHS_BALANCE BETWEEN MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_CURR`) - 12 AND MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_CURR`) THEN 1
        ELSE 0
    END AS ultimos_12_meses_flag
FROM POS_CASH_balance_00
ORDER BY `SK_ID_CURR`, MONTHS_BALANCE;
""")

POS_CASH_balance_01.createOrReplaceTempView("POS_CASH_balance_01")
POS_CASH_balance_01.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10001358

In [16]:
# POS_CASH_balance_01.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## **Criando variáveis explicativas de primeira camada**

- usar média, mínimo, soma, máximo
- considerar variáveis categóricas e variáveis numéricas

### **Gerar variáveis.**

In [25]:
# Importar SparkSession
from pyspark.sql import SparkSession

# Criar uma sessão do Spark
spark = SparkSession.builder.appName("AdaptacaoCodigo").getOrCreate()

df_original_spark = POS_CASH_balance_01

# Inicializar o contador
contador_resultados = 0

# Criar lista para armazenar os resultados
resultados = []

for ag in agg:
    for var_cat in var_categoricas:
        # Filtrar valores distintos não nulos
        distinct_values = df_original_spark.select(var_cat).distinct().na.drop(subset=[var_cat]).rdd.flatMap(lambda x: x).collect()

        for dom_cat_value in distinct_values:
            for var_numerica in var_numericas:
                resultado = f"ROUND({ag}(CASE WHEN {var_cat} = '{dom_cat_value}' THEN {var_numerica} ELSE NULL END), 2) as {ag.lower()}_{var_numerica.lower()}_{var_cat.lower()}_{dom_cat_value.lower().replace(' ', '_')},"

                # Adicionar o resultado à lista
                resultados.append(resultado)

                for mes_flag in meses_flag:
                    resultado = f"ROUND({ag}(CASE WHEN {var_cat} = '{dom_cat_value}' AND {mes_flag} = 1 THEN {var_numerica} ELSE NULL END), 2) as {ag.lower()}_U{mes_flag.split('_')[1]}M_{var_numerica.lower()}_{var_cat.lower()}_{dom_cat_value.lower().replace(' ', '_')},"

                    # Adicionar o resultado à lista
                    resultados.append(resultado)

                    # Incrementar o contador
                    contador_resultados += 1

# Printar o total de combinações
print(f"Total de combinações: {contador_resultados}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total de combina??es: 720

In [26]:
# Inicializa a lista para armazenar os resultados
resultados_formatados = []

# Se há pelo menos um resultado na lista
if resultados:
    # Remove a vírgula do último item
    resultados[-1] = resultados[-1].rstrip(',')

    # Adiciona os resultados formatados à lista
    resultados_formatados.extend(resultados)

    # Une a lista em uma string usando uma quebra de linha como separador
    resultados_str = '\n'.join(resultados_formatados)
    # print(resultados_str)
else:
    resultados_str = "A lista de resultados está vazia."

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### **Colar variáveis criadas no código PySpark-SQL.**

In [27]:
POS_CASH_balance_02 = spark.sql(f'''
SELECT
  SK_ID_CURR as SK_ID_CURR_POS_CASH_balance,
  {resultados_str}
FROM
  POS_CASH_balance_01
GROUP BY
  SK_ID_CURR
''')

POS_CASH_balance_02.createOrReplaceTempView("POS_CASH_balance_02")
POS_CASH_balance_02.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

337252

In [28]:
print(len(POS_CASH_balance_02.columns))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

901

In [29]:
# POS_CASH_balance_02.show(5,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## **Salvando tabela no diretório S3.**

In [30]:
# Save to parquet
path_output = os.path.join('s3://',S3_DATA_OUTPUT_PATH,output)
print(path_output)
POS_CASH_balance_02.coalesce(1).write.\
parquet(path_output,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://pod-academy-analise-de-credito-para-fintech/feature-engineering/02-Previous-Application/abt_POS_CASH_balance

In [23]:
# abt_POS_CASH_balance=sqlContext.read.format("parquet").load('s3://',S3_DATA_OUTPUT_PATH,output)
# abt_POS_CASH_balance.registerTempTable("abt_POS_CASH_balance")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# abt_POS_CASH_balance.show(5,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…